In [62]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np
from time import sleep
from random import randint

In [2]:
url = "https://newyork.craigslist.org/search/sss?query=bikes"
response = requests.get(url)
if response.status_code == 200:
    print("Success")
else:
    print("Failure")

Success


In [63]:
results_page = BeautifulSoup(response.content,'lxml')

# find the total number of posts
results_num = results_page.find('div', class_= 'search-legend')
results_total = int(results_num.find('span', class_='totalcount').text)

prices = []
neighborhood = []
urls = []
dates = []

pages = np.arange(0, results_total+1, 120)

# Loop through all pages
for page in pages:
    
    response = requests.get(url 
                   + "s=" #the parameter for defining the page number 
                   + str(page) #the page number in the pages array from earlier
                   + "&hasPic=1"
                   + "&availabilityMode=0")

    sleep(randint(1,5))

    if response.status_code != 200:
        print(('Request: {}; Status code: {}'.format(requests, response.status_code)))
    
    page_html = BeautifulSoup(response.text, 'html.parser')

    posts = results_page.find_all('li', class_= 'result-row')

    for post in posts:
        prices.append(post.find('span', class_="result-price").text)
        neighborhood.append(post.find('span', class_="result-hood").text)
        urls.append(post.find('a', class_="result-title hdrlnk")['href'])
        dates.append(post.find('time', class_="result-date")['datetime'])

In [64]:
df = pd.DataFrame({'Price':prices, 'Neighborhood':neighborhood, 'URL':urls, 'Date':dates})
df

,Price,Neighborhood,URL,Date
0,$90,(BROOKLYN brooklyn ),https://newyork.craigslist.org/brk/bik/d/brook...,2022-03-03 09:21
1,$50,(BROOKLYN brooklyn ),https://newyork.craigslist.org/brk/bop/d/brook...,2022-03-03 09:20
2,$220,(BROOKLYN brooklyn ),https://newyork.craigslist.org/brk/bik/d/brook...,2022-03-03 09:22
3,$50,(BROOKLYN brooklyn ),https://newyork.craigslist.org/brk/bop/d/brook...,2022-03-03 09:22
4,$200,(greenpoint brooklyn ),https://newyork.craigslist.org/brk/bik/d/brook...,2022-03-03 08:59
...,...,...,...,...
2635,$155,(Queens queens ),https://newyork.craigslist.org/que/bik/d/ridge...,2022-02-25 16:19
2636,$49,(HICKSVILLE long island ),https://newyork.craigslist.org/lgi/hsh/d/bikes...,2022-02-25 15:26
2637,$30,(Greenpoint/Williamsburg Border brooklyn ),https://newyork.craigslist.org/brk/bop/d/brook...,2022-02-25 14:13
2638,$300,(Park Slope brooklyn ),https://newyork.craigslist.org/brk/bik/d/brook...,2022-02-25 12:57
